In [4]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras import optimizers
import argparse
from RNN_utils import *

DATA_DIR = './data/shakespeare_input.txt'
BATCH_SIZE = 30
HIDDEN_DIM = 300
SEQ_LENGTH = 50

lossfile = './data/warpeace_loss.txt'

GENERATE_LENGTH = 100
LAYER_NUM = 2

# Creating training data
X, y, VOCAB_SIZE, ix_to_char, char_to_ix = load_data(DATA_DIR, SEQ_LENGTH)

# Creating and compiling the Network
model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
  model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
rms_prop = optimizers.RMSprop(lr=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=rms_prop)

# Generate some sample before training to know how bad it is!
# print("\n\nGenerating untrained text...\n\n")
# generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char, -1, 0)



Data length: 4573338 characters
Vocabulary size: 67 characters


In [6]:
import keras.callbacks
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

In [7]:
history = model.fit(X, y, batch_size=BATCH_SIZE, validation_split=0.2, epochs=1, verbose=1)

Train on 73172 samples, validate on 18294 samples
Epoch 1/1
73172/73172 [==============================] - 410s 6ms/step - loss: 2.9432 - val_loss: 2.5200


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
history